# Домашняя работа №1

## Подготовка

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import sqlite3
import requests
from tqdm import tqdm

In [2]:
# задаём минимальное и минимальное число обьектов (подсмотрели на сайте) и шаг запросов
min_obj = 1
max_obj = 10534
step = 1000

In [3]:
# объявляем функцию, получающую порцию данных об объектах с offset по limit
def get_chunk(off, lim):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={off}&limit={lim}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json()
    objects_list = objects_data.get('data').get('list')
    result = [obj.get('objId') for obj in objects_list]
    return result

In [4]:
# объявляем функцию, получающую данные о конкретном объекте
def get_obj_info(obj_id):
    res = requests.get(f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{obj_id}')
    return res.json()

## Получение списка объектов

In [5]:
# создаём пустой список объектов
full_objects_list = []

# в цикле получаем данные обо всех объектах порциями по 1000
for offset_, limit_ in tqdm([
    (min_obj + step * x, min_obj + step * (x + 1) if min_obj + step * (x + 1) <= max_obj else max_obj) for x in range(round(max_obj / step))
]):
    full_objects_list += get_chunk(offset_, limit_)

100%|██████████| 11/11 [05:00<00:00, 27.36s/it]


In [6]:
# записываем полный список объектов на диск для дальнейшего использования
with open('full_objects_list.txt', 'w', encoding='utf-8') as f:
    f.writelines(set(str(obj)+'\n' for obj in full_objects_list))

## Получение данных обо всех объектах

In [7]:
# создаём пустой датафрейм для записи данных об объектах
full_data = pd.DataFrame()

In [8]:
# в цикле добавляем к датафрейму сведения о каждом объекте
for obj in tqdm(set(full_objects_list)):
    temp = pd.json_normalize(get_obj_info(obj)['data'])
    full_data = pd.concat(
        [
            full_data,
            temp
        ],
        sort=False
    ).reset_index(drop=True)

100%|██████████| 10534/10534 [38:09<00:00,  4.60it/s] 


In [9]:
# смотрим, что всё в порядке
display(full_data.head(10))

,id,pdId,region,address,nameObj,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,...,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors
0,45592,543495,64,"г Энгельс, ул Трудовая, д. 12, строение 1Г","ЖК ""Рио"" д.1Г",25.0,25.0,154,2024-09-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32787,181636,16,"г Набережные Челны, вдоль проспекта Яшьлек, бл...","ЖК ""Современный""",12.0,13.0,99,2022-12-31,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32793,162963,50,"г Красногорск, район Марьино, корпус 25","Микрогород ""В лесу""",9.0,13.0,545,2023-03-31,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32849,167016,37,"г Кохма, улица Ивановская, литера В, 5 этап","Микрорайон ""Тихий Берег"", 004",8.0,8.0,183,2022-12-31,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32873,141586,42,"г Кемерово, бульвар Строителей, д. 60, корпус 4/1",NaN,12.0,14.0,189,2022-12-31,Кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,32892,117906,50,"с/п Павло-Слободское, улица Красная, д. 9","ЖК ""СКАЗКА"" дом 64",3.0,3.0,22,2022-12-31,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,32893,117906,50,"с/п Павло-Слободское, улица Красная, д. 9","ЖК ""СКАЗКА"" дом 65",3.0,3.0,22,2022-12-31,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,32894,117906,50,"с/п Павло-Слободское, улица Красная, д. 9","ЖК ""СКАЗКА"" дом 66",3.0,3.0,22,2022-12-31,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,32895,153352,64,"г Балаково, улица Титова, блок-секция В, 3-й э...",NaN,11.0,11.0,43,2022-06-30,Другое,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,32897,162604,77,"город Москва, улица Летная, вл. 95Б, блок-секц...",Жилой район ALIA,2.0,27.0,1314,2023-06-30,Монолит-кирпич,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Сохранение данных

In [10]:
# записываем в excel
full_data.to_excel('full_data.xlsx', index=False)

In [11]:
# записываем в pickle
full_data.to_pickle('full_data.pickle')

In [12]:
# записываем в БД
conn =  sqlite3.connect('test_database')
full_data.applymap(lambda c: str(c)).to_sql('products', conn, if_exists='replace', index = False, chunksize=1000)
conn.commit()
conn.close()

## Задание со *

In [13]:
# импортируем необходимые библиотеки
from bs4 import BeautifulSoup
import os

In [14]:
# объявляем функцию для получения изображений объекта
def get_images(obj):
    # загружаем страницу - описание объекта
    response = requests.get(f'https://наш.дом.рф/сервисы/каталог-новостроек/объект/{obj}')

    # создаём парсер HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # если в HTML-коде страницы (установлено путём исследования кода) есть тег <div> с классом swiper-slide,
    # значит на странице есть фотогалерея объекта и есть, что сохранять
    if soup.find_all('div', class_='swiper-slide'):
        # задаём начальное значение счётчика фото
        counter = 0

        # создаём папку с номером объекта, куда будем класть фото
        os.mkdir(f'{obj}')

        # в цикле находим все теги <img>, получаем их атрибут src и загружаем изображения, используя src как url на диск
        for d in soup.find_all('div', class_='swiper-slide'):
            url = d.img['src']
            response = requests.get(url)
            if response.status_code == 200:
                with open(f'{obj}\\{obj}_{counter}.jpg', 'wb') as img_f:
                    img_f.write(response.content)
                counter +=1

        # выводим, сколько изображений успешно скачали
        return f"DOWNLOADED {counter} IMAGES ON OBJECT'S #{obj} PAGE"

    # если галереи на странице нет, пишем, что фото не будет
    else:
        return f"NO IMAGES ON OBJECT'S #{obj} PAGE"

In [15]:
# в цикле вызываем нашу функцию для каждого объекта и скачиваем файлы
for obj in tqdm(list(set(full_objects_list))[:100]):
    print(get_images(obj))

  1%|          | 1/100 [00:05<08:28,  5.14s/it]

DOWNLOADED 2 IMAGES ON OBJECT'S #45592 PAGE


  2%|▏         | 2/100 [00:05<04:14,  2.59s/it]

DOWNLOADED 2 IMAGES ON OBJECT'S #32787 PAGE


  3%|▎         | 3/100 [00:09<04:35,  2.84s/it]

...

DOWNLOADED 1 IMAGES ON OBJECT'S #33187 PAGE


 99%|█████████▉| 99/100 [05:40<00:03,  3.16s/it]

DOWNLOADED 1 IMAGES ON OBJECT'S #33188 PAGE


100%|██████████| 100/100 [05:43<00:00,  3.44s/it]

DOWNLOADED 1 IMAGES ON OBJECT'S #33189 PAGE


❗ вызов сделан только для первых 100 объектов, т.к. по моим оценкам для всех объектов объём фото составляет от 20 до 40 Гб, и качать их смысла нет